Loading Data:

In [501]:
import pandas as pd

# Load the Excel file into a DataFrame
data = pd.read_excel("Survey_ICS487_Project.xlsx")


Data Inspection:

In [502]:
# Check the first few rows
print(data.head())

# Get an overview of the data
print(data.info())

# Check for missing values
print(data.isnull().sum())


    Name  Salary (SAR)  Age    State     Sex  Monthly Debt (SAR)  \
0   Omar          1614   28   Single    Male              277.31   
1   Naif          2325   22  Married  Female              591.89   
2   Saad          2723   48  Married  Female              350.54   
3  Noura         15743   19   Single    Male             1149.78   
4    Ali         22269   30   Single  Female             4885.22   

   Elementary Expenses (SAR)        Goal  Number of Children  \
0                    3911.03  Investment                   0   
1                    6625.49  Investment                   4   
2                    7410.93  Investment                   2   
3                    1996.29    Spending                   0   
4                    2388.83  Investment                   0   

  Employment_Status Nationality Budgeting_Rule Financial_Comfortability  \
0           Student       Other       50/30/20                   ['no']   
1           Student       Saudi       50/30/20          

Clean and Preprocess:

In [503]:
# Standardize column names
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]
# Check for duplicates and remove them
data = data.drop_duplicates()

# Handle missing values (drop rows or fill them as needed)
data = data.dropna()  # Drop rows with missing values
# Alternatively, fill missing values (example: fill salary with median)
data['salary_(sar)'] = data['salary_(sar)'].fillna(data['salary_(sar)'].median())

# Ensure numeric columns are properly formatted
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Validate ranges for numeric columns
data = data[(data['age'] >= 18) & (data['age'] <= 100)]  # Age should be between 0 and 100
data = data[data['salary_(sar)'] >= 0]  # Salary should not be negative

# Filter out rows where the conditions are met
filtered_data = data[
    (data['nationality'].str.lower() == 'saudi') ]  # Keep only rows where nationality is 'saudi'
filtered_data = filtered_data[
    (filtered_data['financial_comfortability'].str.lower() != 'no') ]  # Exclude rows where financial_comfortability is 'no'
filtered_data = filtered_data[
    (data['goal_progress'].str.lower() != 'no')  # Exclude rows where goal_progress is 'no'
]

# Drop the irrelevant columns
filtered_data = filtered_data.drop(columns=['nationality', 'financial_comfortability', 'goal_progress'])

# Store the filtered data back if needed
data = filtered_data
print(data.tail())



        name  salary_(sar)  age    state     sex  monthly_debt_(sar)  \
1170   Malik          2534   20   Single    Male              460.27   
1172  shabab           923   53   Single    Male              166.03   
1173    Azoz          2280   28   Single  Female              474.47   
1174   Ahmed          1168   29  Married    Male               82.45   
1175    Asma         16544   29   Single  Female             4603.31   

      elementary_expenses_(sar)        goal  number_of_children  \
1170                    3232.58     Savings                   0   
1172                    1499.43    Spending                   0   
1173                    4149.74  Investment                   0   
1174                    3948.37    Spending                   2   
1175                    3552.79     Savings                   0   

     employment_status budgeting_rule  
1170           Student       50/30/20  
1172        Unemployed       50/30/20  
1173           Student       50/30/20  
1174

/var/folders/yc/7vmb_7z55z3d92njbdlxxr3m0000gp/T/ipykernel_61078/1195411881.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[


Removing outliers:

In [504]:
import numpy as np

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to numeric columns with potential outliers
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


# Define age bins and labels
age_bins = [0, 24, 34, 44, 54, float('inf')]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']

# Create a new column for age groups
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True)

data = data.drop(columns=['age'])


Encoding Categorical Variables:

In [505]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical columns
categorical_columns = ['state', 'sex', 'goal', 'employment_status']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Encode the age_group using LabelEncoder
label_encoder = LabelEncoder()
data['age_group_encoded'] = label_encoder.fit_transform(data['age_group'])

# Display the encoding
print("Age Group Encoding:")
for label, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{label}: {encoded}")

data = data.drop(columns=['age_group'])


Age Group Encoding:
18-24: 0
25-34: 1
35-44: 2
45-54: 3
55+: 4


Feature Engineering:

In [506]:
# Create new features
data['debt_to_income_ratio'] = data['monthly_debt_(sar)'] / data['salary_(sar)']
data['savings_ratio'] = data['elementary_expenses_(sar)'] / data['salary_(sar)']

# Handle infinite or NaN values due to division
data['debt_to_income_ratio'] = data['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)
data['savings_ratio'] = data['savings_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop irrelevant columns
data = data.drop(columns=['name'])

data.describe()
data.head()

,salary_(sar),monthly_debt_(sar),elementary_expenses_(sar),number_of_children,budgeting_rule,state_Single,sex_Male,goal_Savings,goal_Spending,employment_status_Student,employment_status_Unemployed,age_group_encoded,debt_to_income_ratio,savings_ratio
1,2325,591.89,6625.49,4,50/30/20,False,False,False,False,True,False,0,0.254576,2.849673
3,15743,1149.78,1996.29,0,60/20/20,True,True,False,True,False,False,0,0.073034,0.126805
5,17615,206.64,3237.91,0,70/20/10,True,True,False,False,False,False,3,0.011731,0.183815
7,1554,88.62,10180.03,3,50/30/20,False,True,False,True,False,True,0,0.057027,6.550856
8,16069,2073.30,3038.13,1,50/30/20,False,True,False,False,False,False,2,0.129025,0.189068


Saving Data:

In [507]:
# Save cleaned data to a new Excel file
data.to_excel("Cleaned_Survey_ICS487_Project.xlsx", index=False)

# Or save it as a CSV file
data.to_csv("Cleaned_Survey_ICS487_Project.csv", index=False)